## Importing libraries

In [ ]:
from collections import Counter

## Obtaining the results on the validation set

Sending the model to CPU

In [ ]:
model.to('cpu')

Getting the model answers to evaluate

In [ ]:
eval_answers = []

for i in range(len(val_questions)):
  question, text = val_questions[i], val_contexts[i]

  input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"

  input_ids = tokenizer.encode(input_text)
  token_type_ids = [0 if i <= input_ids.index(102) else 1
      for i in range(len(input_ids))]

  start_scores, end_scores = model(torch.tensor([input_ids]), \
      token_type_ids=torch.tensor([token_type_ids]))

  all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  ans = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

  eval_answers.append(ans)

Computing Precision, Recall and F1-Score

In [ ]:
num_c = []
num_p = []
num_g = []

for a in range(len(eval_answers)):

  common = collections.Counter(val_answers[a].split()) & collections.Counter(eval_answers[a].split()) # tokens shared between gold and predicted answers
  num_common = sum(common.values())

  num_pred = len(str(eval_answers[a]).split()) # the number of predicted tokens

  num_gold = len(str(val_answers[a]).split()) # the number of gold tokens

  num_c.append(num_common)
  num_p.append(num_pred)
  num_g.append(num_gold)

precision = 1.0 * sum(num_c) / sum(num_p) # the num of tokens shared between gold and predicted answers / the num of predicted tokens
recall = 1.0 * sum(num_c) / sum(num_g) # the num of tokens shared between gold and predicted answers / the num of gold tokens
f1_score = (2 * precision * recall) / (precision + recall)